In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main-2'

In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)



In [2]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(83000, 502) (21975, 502)


In [3]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((83000, 502), (10987, 502), (10988, 502))

In [4]:
import torch
import torch.utils.data

#drop rows
test.drop(test.tail(37).index,inplace = True)
val.drop(val.tail(38).index,inplace = True)
test.shape, val.shape


# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10950, 1])


In [5]:
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class QRNNLayer(nn.Module):
    def __init__(self,batch_size,input_size,n_filters,kernel_size,embed_size,device,dropout):
        super(QRNNLayer,self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.embed_size = embed_size
        self.dropout = torch.nn.Dropout(dropout)
        self.device = device
        self.conv1 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
        self.conv2 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
        self.conv3 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
    
    def forward(self,masked_input, h, c):
        Z,F,O = self.masked_conv(masked_input)
        h, c = self.pool(c,Z,F,O)
        masked_input = h
        return masked_input,h,c
    
    def masked_conv(self,x):
        pad = torch.zeros([self.batch_size,1,self.input_size],device=self.device)
        x = torch.cat([pad,x],1).permute(0,2,1)
        Z = torch.tanh((self.conv1(x)))
        F = torch.sigmoid((self.conv2(x)))
        O = torch.sigmoid((self.conv3(x)))
        one_mask = torch.ones_like(F,device=self.device) - F
        F = 1 - self.dropout(one_mask)
        return Z.permute(0,2,1), F.permute(0,2,1), O.permute(0,2,1)
    
    def pool(self, prev_c,Z,F,O):
        c = torch.mul(F,prev_c) + torch.mul(1-F,Z)
        h = torch.mul(O,c)
        return h,c

class QRNN(nn.Module):
    def __init__(self,trial,vocab_size,embed_size,n_filters,kernel_size,batch_size,seq_len,layers,device,dropout):
        super(QRNN,self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        self.num_layer = trial.suggest_int("num_layer",1,4)
        self.embed_size = trial.suggest_int("embed_size",16,128)
        self.n_filters = trial.suggest_int("n_filters",50,200)
        # self.kernel_size = trial.suggest_int("kernel_size",2,5)
        # self.embed_size = embed_size
        # self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        # self.num_layer = layers
        self.device = device
        self.embedding = torch.nn.Embedding(vocab_size, self.embed_size)
        self.dense = torch.nn.Linear(self.seq_len*self.n_filters,1)
        self.QRNN_layers = torch.nn.ModuleList([QRNNLayer(self.batch_size,self.embed_size if l==0 else self.n_filters,
                                                         self.n_filters,self.kernel_size,self.embed_size,self.device,
                                                         dropout,) for l in range(self.num_layer)])
        
        
    def forward(self, x, target):
        x = self.embedding(x)
        h = torch.zeros([self.batch_size,self.seq_len,self.n_filters],device=self.device)
        c = torch.zeros_like(h,device=self.device)
        
        masked_input = x
        for l,layer in enumerate(self.QRNN_layers):
            masked_input,h,c = layer(masked_input,h,c)
        dense_input = h.reshape([self.batch_size,-1])
        logits = self.dense(dense_input)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy

In [6]:
data_dir = './qrnn-amazon-hyperband-trails_10/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./qrnn-amazon-hyperband-trails_10/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [7]:
import time
epochs = 10
n_filters = 64
kernel_size = 2
layers= 2
learning_rate = 0.001
print(len(word_dict))
vocab_size = 10000
print(vocab_size)
embed_dims = 32
seq_len = 500
dropout = 0.3
batch_size=50

9998
10000


In [8]:
import time
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.optim as optim
import numpy as np

# Define an objective function to be minimized.
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #epochs = trial.suggest_int("epochs",5,15)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = QRNN(trial,vocab_size, embed_dims, n_filters, kernel_size, batch_size, seq_len, layers, device, dropout).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss()
    counter = 0
    QRNN_acc = []
    QRNN_valacc = []
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_acc_epoch = []
    time_epoch = []
    final_val_acc = 0
    model.train()
    for e in range(epochs):
        start_time = time.time()
        train_loss = []
        train_acc = []
        for inputs, labels in train_dl:
            inputs, labels = inputs.cuda(), labels.cuda()
            model.zero_grad()
            logits, accuracy = model(inputs,labels)
            loss = criterion(logits,labels.float())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            train_loss.append(loss.item())
            train_acc.append(accuracy.item()*100/batch_size)
            if counter%400==0:
                print("Epoch: {}/{}".format(e,epochs),
                             "\tIteration: {}".format(counter),
                             "\tTrain Loss: {:.3f}".format(loss.item()),
                             "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
                QRNN_acc.append(accuracy.item()*100/batch_size)
            counter += 1
        train_loss_epoch.append(np.round(np.mean(train_loss), 3))
        train_acc_epoch.append(np.round(np.mean(train_acc), 3))
        print("\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))
        
        with torch.no_grad():
            model.eval()
            val_acc = []
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.cuda(), labels.cuda()
                logits_val,accuracy_val = model(inputs_val,labels_val)
                loss_val = criterion(logits_val,labels_val.float())
                val_acc.append(accuracy_val.item()*100/batch_size)
                val_loss.append(loss_val.item())
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(np.mean(val_acc)))
            val_loss_epoch.append(np.round(np.mean(val_loss), 3))
            val_acc_epoch.append(np.round(np.mean(val_acc), 3))
            QRNN_valacc.append(np.mean(val_acc))
            model.train()
            final_val_acc = np.mean(val_acc)
            end_time = time.time()-start_time
            print("Time to train epoch: {0} s".format(end_time))
            time_epoch.append(np.round(end_time, 3))
    write_to_csv(trial.number, epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
    return final_val_acc


In [9]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [10]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=10,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 22:54:19,301] A new study created in memory with name: no-name-b93d9e43-3c60-45d9-bc28-174f3fa57927


Epoch: 0/10 	Iteration: 0 	Train Loss: 0.684 	Train Accuracy: 68.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 0.494 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 800 	Train Loss: 0.375 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 1200 	Train Loss: 0.313 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 1600 	Train Loss: 0.348 	Train Accuracy: 84.00
	Train Loss: 0.445 	Train Acc: 79.967
Val Loss: 0.376 	Val Acc: 83.644
Time to train epoch: 26.873484134674072 s
Epoch: 1/10 	Iteration: 2000 	Train Loss: 0.285 	Train Accuracy: 90.00
Epoch: 1/10 	Iteration: 2400 	Train Loss: 0.305 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 2800 	Train Loss: 0.314 	Train Accuracy: 88.00
Epoch: 1/10 	Iteration: 3200 	Train Loss: 0.417 	Train Accuracy: 80.00
	Train Loss: 0.337 	Train Acc: 85.289
Val Loss: 0.335 	Val Acc: 85.854
Time to train epoch: 26.11928629875183 s
Epoch: 2/10 	Iteration: 3600 	Train Loss: 0.258 	Train Accuracy: 92.00
Epoch: 2/10 	Iteration: 4000 	Train Loss: 0.321 	Train Accuracy: 82.00

[I 2022-05-14 22:58:53,680] Trial 0 finished with value: 87.71689497716895 and parameters: {'optimizer': 'Adam', 'lr': 0.00021204123434779076, 'dropout': 0.24971384016508344, 'num_layer': 2, 'embed_size': 37, 'n_filters': 157}. Best is trial 0 with value: 87.71689497716895.


Val Loss: 0.312 	Val Acc: 87.717
Time to train epoch: 26.108797311782837 s
Epoch: 0/10 	Iteration: 0 	Train Loss: 0.697 	Train Accuracy: 32.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 0.535 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 800 	Train Loss: 0.512 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 1200 	Train Loss: 0.501 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 1600 	Train Loss: 0.575 	Train Accuracy: 74.00
	Train Loss: 0.546 	Train Acc: 76.402
Val Loss: 0.549 	Val Acc: 75.744
Time to train epoch: 68.54233288764954 s
Epoch: 1/10 	Iteration: 2000 	Train Loss: 0.483 	Train Accuracy: 82.00
Epoch: 1/10 	Iteration: 2400 	Train Loss: 0.664 	Train Accuracy: 66.00
Epoch: 1/10 	Iteration: 2800 	Train Loss: 0.473 	Train Accuracy: 84.00
Epoch: 1/10 	Iteration: 3200 	Train Loss: 0.547 	Train Accuracy: 76.00
	Train Loss: 0.536 	Train Acc: 76.431
Val Loss: 0.531 	Val Acc: 75.744
Time to train epoch: 68.57172226905823 s
Epoch: 2/10 	Iteration: 3600 	Train Loss: 0.431 	Train Accuracy: 86

[I 2022-05-14 23:10:19,293] Trial 1 finished with value: 86.63013698630137 and parameters: {'optimizer': 'SGD', 'lr': 0.03638114417898433, 'dropout': 0.26038280435292016, 'num_layer': 4, 'embed_size': 107, 'n_filters': 191}. Best is trial 0 with value: 87.71689497716895.


...

Val Loss: 24.256 	Val Acc: 75.744
Time to train epoch: 64.53000068664551 s
Epoch: 0/10 	Iteration: 0 	Train Loss: 0.697 	Train Accuracy: 34.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 0.540 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 800 	Train Loss: 0.512 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 1200 	Train Loss: 0.501 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 1600 	Train Loss: 0.578 	Train Accuracy: 74.00
	Train Loss: 0.547 	Train Acc: 76.404
Val Loss: 0.552 	Val Acc: 75.744
Time to train epoch: 50.59541630744934 s
Epoch: 1/10 	Iteration: 2000 	Train Loss: 0.487 	Train Accuracy: 82.00
Epoch: 1/10 	Iteration: 2400 	Train Loss: 0.663 	Train Accuracy: 66.00
Epoch: 1/10 	Iteration: 2800 	Train Loss: 0.467 	Train Accuracy: 84.00
Epoch: 1/10 	Iteration: 3200 	Train Loss: 0.551 	Train Accuracy: 76.00
	Train Loss: 0.544 	Train Acc: 76.431
Val Loss: 0.551 	Val Acc: 75.744
Time to train epoch: 50.62818646430969 s
Epoch: 2/10 	Iteration: 3600 	Train Loss: 0.429 	Train Accuracy: 86

[I 2022-05-14 23:55:55,732] Trial 8 finished with value: 75.74429223744292 and parameters: {'optimizer': 'SGD', 'lr': 0.005201238151384301, 'dropout': 0.2768168872674762, 'num_layer': 3, 'embed_size': 62, 'n_filters': 199}. Best is trial 3 with value: 88.10045662100457.


Val Loss: 0.524 	Val Acc: 75.744
Time to train epoch: 50.609801054000854 s
Epoch: 0/10 	Iteration: 0 	Train Loss: 0.685 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 0.259 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 800 	Train Loss: 0.284 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 1200 	Train Loss: 0.269 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 1600 	Train Loss: 0.257 	Train Accuracy: 94.00
	Train Loss: 0.353 	Train Acc: 84.758
Val Loss: 0.299 	Val Acc: 87.224
Time to train epoch: 19.0924015045166 s
Epoch: 1/10 	Iteration: 2000 	Train Loss: 0.324 	Train Accuracy: 80.00
Epoch: 1/10 	Iteration: 2400 	Train Loss: 0.210 	Train Accuracy: 92.00
Epoch: 1/10 	Iteration: 2800 	Train Loss: 0.289 	Train Accuracy: 88.00
Epoch: 1/10 	Iteration: 3200 	Train Loss: 0.382 	Train Accuracy: 80.00
	Train Loss: 0.264 	Train Acc: 88.990
Val Loss: 0.289 	Val Acc: 87.927
Time to train epoch: 19.08648371696472 s
Epoch: 2/10 	Iteration: 3600 	Train Loss: 0.140 	Train Accuracy: 96.

[I 2022-05-14 23:59:06,466] Trial 9 finished with value: 87.72602739726027 and parameters: {'optimizer': 'Adam', 'lr': 0.0011878682391953538, 'dropout': 0.20920277987028832, 'num_layer': 3, 'embed_size': 51, 'n_filters': 71}. Best is trial 3 with value: 88.10045662100457.


Val Loss: 0.569 	Val Acc: 87.726
Time to train epoch: 19.07549238204956 s
Study statistics: 
  Number of finished trials:  10
  Number of complete trials:  10
Best trial:
  Value:  88.10045662100457
  Params: 
    optimizer: Adam
    lr: 0.00011567125810289309
    dropout: 0.2400100007797182
    num_layer: 4
    embed_size: 52
    n_filters: 176


In [15]:
data_dir = './qrnn-amazon-hyperband/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)
    
torch.save(best_model.state_dict(),"./qrnn-amazon-hyperband/qrnn-imdb-hyperband_10.pth")

In [16]:
criterion = torch.nn.BCELoss()
def test(model, test_dl, epochs):
    with torch.no_grad():
        model.eval()
        test_acc = []
        test_loss = []
        for inputs, labels in test_dl:
            input_test, labels_test = inputs.cuda(), labels.cuda()
            logits_test, accuracy_test = model(input_test, labels_test)
            loss_test = criterion(logits_test, labels_test.float())
            test_acc.append(accuracy_test.item()*100/batch_size)
            test_loss.append(loss_test.item())
        print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))

In [17]:
test(best_model, test_dl, epochs)

Test Loss: 0.278 	Test Acc: 88.338
